# Generate a Noise Model using Bootstrapping

Here we assume that we do not have access to calibration data to create a noise model for training DivNoising. In this case, we use an approach called ```Bootstrapping``` to create a noise model from noisy data itself. The idea is that we will first use the unsupervised denoising method Noise2Void to obtain denoised images corresponding to our noisy data. Then we will treat the denoised images as pseudo GT corresponding to the noisy data and use the pair of noisy images and corresponding Noise2Void denoised images to learn a noise model.

DivNoising when using bootstrapped noise model generally gives better results compared to Noise2Void denoising. Also, unlike Noise2Void, we additionally obtain diverse denoised samples corresponding to any noisy image unlike Noise2Void.

__Note:__ Denoising methods other than Noise2Void can also be used to obtain pseudo GT for bootsrapping a noise model.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import torch
import os
import urllib
import zipfile
from torch.distributions import normal
import matplotlib.pyplot as plt, numpy as np, pickle
from scipy.stats import norm
from tifffile import imread
import sys
sys.path.append('../../')
from divnoising.gaussianMixtureNoiseModel import GaussianMixtureNoiseModel
from divnoising import histNoiseModel
from divnoising.utils import plotProbabilityDistribution

dtype = torch.float
device = torch.device("cuda:0") 

### Download data

Download the data from https://zenodo.org/record/5156913/files/Convallaria_diaphragm.zip?download=1. Here we show the pipeline for Convallaria dataset. Save the dataset in an appropriate path. For us, the path is the data folder which exists at `./data`.

In [ ]:
# Download data
if not os.path.isdir('./data'):
    os.mkdir('./data')

zipPath="./data/Convallaria_diaphragm.zip"
if not os.path.exists(zipPath):  
    data = urllib.request.urlretrieve('https://zenodo.org/record/5156913/files/Convallaria_diaphragm.zip?download=1', zipPath)
    with zipfile.ZipFile(zipPath, 'r') as zip_ref:
        zip_ref.extractall("./data")

In [ ]:
observation= imread('./data/Convallaria_diaphragm/20190520_tl_25um_50msec_05pc_488_130EM_Conv.tif') #Load the noisy data to be denoised

### Load pseudo GT

As described above, we will use the denoising results obtained by Noise2Void and treat them as pseudo GT corresponding to our noisy data. Following this, we will use the pair of noisy images and corresponding Noise2Void denoised images to learn a noise model. You can use any other denoising method as well and treat their denoised result as pseudo GT to learn a noise model for DivNoising training.

If you have access to pseudo GT (denoised images from some other denoising method), provide the directory path for these images in ```pseudo_gt_path``` parameter. If you do not have such pseudo GT, first generate these images by running any denoising method on your data. For example, you can use Noise2Void denoising as shown [here](https://github.com/juglab/n2v).

Next, specify the directory path (```noisy_input_path```) to the noisy data that you wish to denoise with DivNoising.

Using these, we can either bin the noisy - pseudo GT pairs as a 2-D histogram or fit a GMM distribution to obtain a smooth, parametric description of the noise model.

In [ ]:
pseudo_gt_path="./pseudo_gt/"
signal = imread(pseudo_gt_path+'*.tif') # Load pseudo GT (obtained as a result of denoising from other methods)

Specify ```path``` where the noise model will be stored, ```dataName``` for the name of the noise model,  ```n_gaussian``` to indicate how mamny Gaussians willbe used for learning a GMM based noise model, ```n_coeff``` for indicating a polynomial of what degree will be used to patrametrize the parameters of GMM noise model. The default settings for ```n_gaussian``` and ```n_coeff``` generally work well for most datasets.

In [ ]:
path = './data/Convallaria_diaphragm/'
dataName = 'convallaria' # Name of the noise model 
n_gaussian = 3 # Number of gaussians to use for Gaussian Mixture Model
n_coeff = 2 # No. of polynomial coefficients for parameterizing the mean, standard deviation and weight of Gaussian components.

In [ ]:
nameHistNoiseModel ='HistNoiseModel_'+dataName+'_'+'bootstrap'
nameGMMNoiseModel = 'GMMNoiseModel_'+dataName+'_'+str(n_gaussian)+'_'+str(n_coeff)+'_'+'bootstrap'

In [ ]:
# Let's look the raw data and our pseudo ground truth signal
print(signal.shape)
plt.figure(figsize=(12, 12))
plt.subplot(1, 2, 2)
plt.title(label='pseudo ground truth')
plt.imshow(signal[0],cmap='gray')
plt.subplot(1, 2, 1)
plt.title(label='single raw image')
plt.imshow(observation[0],cmap='gray')
plt.show()

### Creating the Histogram Noise Model

Using the raw pixels $x_i$, and our averaged GT $s_i$, we are now learning a histogram based noise model. It describes the distribution $p(x_i|s_i)$ for each $s_i$. 

In [ ]:
# We set the range of values we want to cover with our model.
# The pixel intensities in the images you want to denoise have to lie within this range.
minVal, maxVal = 234, 7402
bins = 256

# We are creating the histogram.
# This can take a minute.
histogram = histNoiseModel.createHistogram(bins, minVal, maxVal, observation,signal)

# Saving histogram to disc.
np.save(path+nameHistNoiseModel+'.npy', histogram)
histogramFD=histogram[0]

In [ ]:
# Let's look at the histogram-based noise model.
plt.xlabel('Observation Bin')
plt.ylabel('Signal Bin')
plt.imshow(histogramFD**0.25, cmap='gray')
plt.show()

### Creating the GMM noise model
Using the raw pixels $x_i$, and our averaged GT $s_i$, we are now learning a GMM based noise model. It describes the distribution $p(x_i|s_i)$ for each $s_i$. 

In [ ]:
min_signal=np.percentile(signal, 0.5)
max_signal=np.percentile(signal, 99.5)
print("Minimum Signal Intensity is", min_signal)
print("Maximum Signal Intensity is", max_signal)

In [ ]:
min_signal=np.min(signal)
max_signal=np.max(signal)
print("Minimum Signal Intensity is", min_signal)
print("Maximum Signal Intensity is", max_signal)

Iterating the noise model training for `n_epoch=4000` and `batchSize=25000` works the best for `Mouse nuclei` dataset. 

In [ ]:
gaussianMixtureNoiseModel = GaussianMixtureNoiseModel(min_signal = min_signal, max_signal =max_signal, 
                                                      path=path, weight = None, n_gaussian = n_gaussian, 
                                                      n_coeff = n_coeff, min_sigma = 50, device = device)

In [ ]:
gaussianMixtureNoiseModel.train(signal, observation, batchSize = 250000, n_epochs = 2000, learning_rate=0.1,
                                name = nameGMMNoiseModel, lowerClip = 0.5, upperClip = 99.5)

### Visualizing the Histogram-based and GMM-based noise models

In [ ]:
plotProbabilityDistribution(signalBinIndex=170, histogram=histogramFD, 
                            gaussianMixtureNoiseModel=gaussianMixtureNoiseModel, min_signal=minVal, 
                            max_signal=maxVal, n_bin= bins, device=device)